In [1]:
import pandas as pd
import io
import re
import os

from stops_prn_extractor import StopsPRNExtractor

# Instantiate the extractor object
extractor = StopsPRNExtractor()


In [2]:
# Instantiate the extractor object
extractor = StopsPRNExtractor()

prn_files_directory = "./stops_output_prn_report_files"

prn_files = [
    ("2024", f"{prn_files_directory}/A2_MBTA-CATA-MWRTA-BATA-MVRTA#MBTA-CATA-MWRTA-BATA-MVRTA#MBTA-CATA-MWRTA-BATA-MVRTA_STOPSY2024Results.prn"),
    ("2045", f"{prn_files_directory}/A2_MBTA-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA_STOPSY2045Results.prn"),
    ("2050", f"{prn_files_directory}/A2_MBTA-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA#MBTA50-CATA-MWRTA-BATA-MVRTA_STOPSY2050Results.prn"),
    # Add more as needed
]

# Add files to the extractor
extractor.add_files(prn_files)

extractor.extract_table_9_01()
extractor.extract_table_10_01() 
# extractor.extract_table_10_02() 

# Display results (optional, good for debugging)
for alias_key in extractor.tables:
    print(f"\nAlias: {alias_key}")
    print("Metadata:")
    for k, v in extractor.metadata[alias_key].items():
        print(f"  {k}: {v}")
    print(f"\nTable data for {alias_key}:")
    display(extractor.tables[alias_key])

# Export extracted tables to CSV files
extractor.export_to_csv(output_dir="./extracted_csv_tables")


Alias: 2024_table_9_01
Metadata:
  Run: Boston_Regional_STOPS                                                                                            10:48:37
  Program: STOPS

Table data for 2024_table_9_01:


,Stop_id1,Station_Name,WLK,Y2024_EXI_KNR,STING_PNR,STING_XFR,STING_ALL,STING_WLK,Y2024_NO_KNR,-BUILD_PNR,-BUILD_XFR,-BUILD_ALL,-BUILD_WLK,Y2024_B_KNR,UILD_PNR,UILD_XFR,UILD_ALL
0,***** AVG WEEKDAY STATION,UTILIZATION (All Tra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Comparison of Station Boa,rdings by Scenario fo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,All Purposes All Transit,All car HH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11334,A6639&T,Assembly,2163,30,0,0,2192,2163,30,0,0,2192,2163,30,0,0,2192
11335,A6428&T,Wellington,1921,367,1630,902,4819,1921,367,1630,902,4819,1921,367,1630,902,4819
11336,A6430&T,Malden Center,5680,487,5,2743,8916,5680,487,5,2743,8916,5680,487,5,2743,8916
11337,A6432&T,Oak Grove,2073,520,51,785,3429,2073,520,51,785,3429,2073,520,51,785,3429



Alias: 2045_table_9_01
Metadata:
  Run: Boston_Regional_STOPS                                                                                            16:58:08
  Program: STOPS

Table data for 2045_table_9_01:


,Stop_id1,Station_Name,WLK,Y2024_EXI_KNR,STING_PNR,STING_XFR,STING_ALL,STING_WLK,Y2045_NO_KNR,-BUILD_PNR,-BUILD_XFR,-BUILD_ALL,-BUILD_WLK,Y2045_B_KNR,UILD_PNR,UILD_XFR,UILD_ALL
0,***** AVG WEEKDAY STATION,UTILIZATION (All Tra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Comparison of Station Boa,rdings by Scenario fo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,All Purposes All Transit,All car HH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11334,A6639&T,Assembly,2163,30,0,0,2192,3272,26,0,217,3515,3272,26,0,217,3515
11335,A6428&T,Wellington,1921,367,1630,902,4819,2221,479,3077,1434,7211,2221,479,3077,1434,7211
11336,A6430&T,Malden Center,5680,487,5,2743,8916,6948,546,3,7400,14897,6948,546,3,7400,14897
11337,A6432&T,Oak Grove,2073,520,51,785,3429,1916,560,8,256,2740,1916,560,8,256,2740



Alias: 2050_table_9_01
Metadata:
  Run: Boston_Regional_STOPS                                                                                            15:01:23
  Program: STOPS

Table data for 2050_table_9_01:


...


Alias: 2045
Metadata:
  Run: Boston_Regional_STOPS                                                                                            16:58:08
  Program: STOPS

Table data for 2045:


,Route_ID_--Route_Name,Count,Y2024_EXI_WLK,STING_KNR,STING_PNR,STING_ALL,Y2045_NO-_WLK,BUILD_KNR,BUILD_PNR,BUILD_ALL,Y2045_BUI_WLK,LD_KNR,LD_PNR,LD_ALL
0,***** AVG WEEKDAY ROUTE UTILIZATION ZONE BY (P...,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Comparison of Route Boardings by Scenario and ...,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Total Transit Trips,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,Mattapan&T --Mattapan Trolley-Ma...,4523,3416,289,896,4601,4871,341,1660,6872,4871,341,1660,6872
258,NCS&M --Natick Commuter Shu...,0,8,0,1,10,8,0,1,10,8,0,1,10
259,Orange&T --Orange Line-Orange ...,115769,93982,8294,12794,115071,117088,10381,26508,153977,117088,10381,26508,153977
260,Red&T --Red Line-Red Line-R...,133995,109776,8110,12642,130528,126546,9829,25890,162266,126546,9829,25890,162266



Alias: 2050
Metadata:
  Run: Boston_Regional_STOPS                                                                                            15:01:23
  Program: STOPS

Table data for 2050:


,Route_ID_--Route_Name,Count,Y2024_EXI_WLK,STING_KNR,STING_PNR,STING_ALL,Y2050_NO-_WLK,BUILD_KNR,BUILD_PNR,BUILD_ALL,Y2050_BUI_WLK,LD_KNR,LD_PNR,LD_ALL
0,***** AVG WEEKDAY ROUTE UTILIZATION ZONE BY (P...,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Comparison of Route Boardings by Scenario and ...,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Total Transit Trips,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,Mattapan&T --Mattapan Trolley-Ma...,4523,3416,289,896,4601,4884,349,1654,6887,4884,349,1654,6887
258,NCS&M --Natick Commuter Shu...,0,8,0,1,10,8,0,1,10,8,0,1,10
259,Orange&T --Orange Line-Orange ...,115769,93982,8294,12794,115071,117748,10476,26393,154617,117748,10476,26393,154617
260,Red&T --Red Line-Red Line-R...,133995,109776,8110,12642,130528,128384,9948,25781,164113,128384,9948,25781,164113


Exported '2024_table_9_01' to './extracted_csv_tables\2024_Table_9_01.csv' successfully.
Exported '2045_table_9_01' to './extracted_csv_tables\2045_Table_9_01.csv' successfully.
Exported '2050_table_9_01' to './extracted_csv_tables\2050_Table_9_01.csv' successfully.
Exported '2024' to './extracted_csv_tables\2024_Table_10_01.csv' successfully.
Exported '2045' to './extracted_csv_tables\2045_Table_10_01.csv' successfully.
Exported '2050' to './extracted_csv_tables\2050_Table_10_01.csv' successfully.
